In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [1]:
 #
 !git clone https://github.com/indrani-16/greenai.git

Cloning into 'greenai'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 122 (delta 15), reused 1 (delta 1), pack-reused 100 (from 1)
Receiving objects: 100% (122/122), 3.61 MiB | 21.64 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [5]:
import pandas as pd
import glob
import os
import zipfile

# Set extraction directory
extract_dir = "5M_trees"

# Check if directory exists
if not os.path.exists(extract_dir):
    raise FileNotFoundError(f"Directory {extract_dir} does not exist. Please check the path.")

# Define which columns to keep
selected_columns = [
    'common_name', 'scientific_name', 'city', 'state',
    'longitude_coordinate', 'latitude_coordinate', 'address', 'condition',
    'native', 'height_binned_M', 'diameter_breast_height_binned_CM',
    'location_type', 'zipcode', 'neighborhood', 'location_name', 'ward',
    'district', 'overhead_utility', 'diameter_breast_height_CM', 'height_M'
]

# Merge all CSVs except metadata
exclude_files = {'Column_Headers_Dryad.csv', 'README_Dryad.txt'}
csv_files = [f for f in glob.glob(os.path.join(extract_dir, "*.csv")) if os.path.basename(f) not in exclude_files]

# Debug: Print found files
print(f"Found CSV files: {csv_files}")

df_list = []
for file in csv_files:
    df = pd.read_csv(file, low_memory=False)

    # Intersect the file's columns with selected_columns
    available_cols = list(set(df.columns) & set(selected_columns))

    # Add any missing columns with NaN to maintain uniformity
    for col in selected_columns:
        if col not in df.columns:
            df[col] = pd.NA

    filtered_df = df[selected_columns].copy()
    df_list.append(filtered_df)

# Check if df_list is empty before concatenation
if not df_list:
    raise ValueError("No valid CSV files found to concatenate. Please check the directory or file names.")

# Merge all filtered data
merged_df = pd.concat(df_list, ignore_index=True)

# Add a unique tree ID
merged_df.insert(0, 'tree_id', ['tree_' + str(i) for i in range(1, len(merged_df) + 1)])

# Check for missing values
print(merged_df.isnull().sum())

Found CSV files: ['/content/drive/MyDrive/5M_trees/SanFrancisco_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/Ontario_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/Sacramento_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/SantaRosa_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/CapeCoral_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/RanchoCucamonga_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/NewOrleans_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/SiouxFalls_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/Irvine_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/Jerseycity_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/Louisville_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/Fresno_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/Minneapolis_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/Seattle_Final_2022-06-18.csv', '/content/drive/MyDrive/5M_trees/Worcester_Final_202

In [6]:
# Drop columns with more than 3,038,500 missing values
threshold = 3038501
merged_df = merged_df.loc[:, merged_df.isnull().sum() <= threshold]

In [7]:
merged_df = merged_df.drop(columns=['diameter_breast_height_binned_CM'])


In [8]:
merged_df = merged_df.dropna(subset=[
    'common_name',
    'scientific_name',
    'longitude_coordinate',
    'latitude_coordinate',
    'condition',
    'diameter_breast_height_CM','address', 'city'
])

In [9]:
merged_df.isnull().sum()

,0
tree_id,0
common_name,0
scientific_name,0
city,0
state,0
longitude_coordinate,0
latitude_coordinate,0
address,0
condition,0
native,0


In [12]:
import pandas as pd

# Load the CSV file
merged_df = pd.DataFrame({
    'common_name': ['Oak', 'Maple', 'Oak', 'Pine', 'Maple', ' ', None, 'Birch', 'Birch']
})
  # make sure this file exists in your working directory


In [13]:



# Step 1: Count species and filter those with at least 2 occurrences
species_counts = merged_df['common_name'].value_counts()
valid_species = species_counts[species_counts >= 2].index.tolist()

# Step 2: Filter valid and non-null species
filtered_df = merged_df[
    merged_df['common_name'].isin(valid_species) &
    merged_df['common_name'].notna()
].copy()

# Step 3: Remove empty strings (after stripping whitespace)
filtered_df = filtered_df[filtered_df['common_name'].str.strip() != ''].copy()

# Step 4: Sanity check
assert filtered_df['common_name'].value_counts().min() >= 2, "Still has species with < 2 samples!"

# Final cleaned data
data = filtered_df.copy()

# Output value counts
print(data['common_name'].value_counts())


common_name
Oak      2
Maple    2
Birch    2
Name: count, dtype: int64


In [22]:


from google.colab import files
uploaded = files.upload()  # Select your CSV file manually

import pandas as pd
df = pd.read_csv("your_uploaded_file.csv")
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from collections import Counter
df = pd.read_csv("5M_trees_cleaned.csv")

# Load the data (Replace with your actual filtered DataFrame or CSV)
df = pd.read_csv("trees.csv")  # or use `filtered_df.copy()` if already defined

# Optional: Simplify scientific name to genus
df['genus'] = df['scientific_name'].apply(lambda x: x.split()[0] if pd.notnull(x) else '')

# Encode categorical variables (ensure no NaNs)
df['native_encoded'] = df['native'].astype('category').cat.codes
df['city_encoded'] = df['city'].astype('category').cat.codes
df['state_encoded'] = df['state'].astype('category').cat.codes

# Features to use for the model
feature_cols = [
    'latitude_coordinate',
    'longitude_coordinate',
    'diameter_breast_height_CM',
    'native_encoded',
    'city_encoded',
    'state_encoded'
]

# Prepare feature matrix
X = df[feature_cols]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the Nearest Neighbors model
nn_model = NearestNeighbors(n_neighbors=50, algorithm='ball_tree')
nn_model.fit(X_scaled)

# Recommender function
def recommend_species(lat, lon, diameter_cm, native, city, state, top_n=5):
    # Encode the input values based on existing categories
    try:
        native_code = df['native'].astype('category').cat.categories.get_loc(native)
        city_code = df['city'].astype('category').cat.categories.get_loc(city)
        state_code = df['state'].astype('category').cat.categories.get_loc(state)
    except KeyError as e:
        return f"Value not found in dataset categories: {e}"

    # Prepare and scale input
    input_features = np.array([[lat, lon, diameter_cm, native_code, city_code, state_code]])
    input_scaled = scaler.transform(input_features)

    # Find nearest neighbors
    distances, indices = nn_model.kneighbors(input_scaled)

    # Count species among neighbors
    neighbors = df.iloc[indices[0]]
    species_counts = Counter(neighbors['common_name'])

    # Return top N recommended species
    return species_counts.most_common(top_n)

# Example usage (modify values as needed)
recommendations = recommend_species(
    lat=37.77,
    lon=-122.42,
    diameter_cm=30,
    native='Yes',
    city='San Francisco',
    state='California',
    top_n=5
)

print("Recommended species:", recommendations)


FileNotFoundError: [Errno 2] No such file or directory: 'your_uploaded_file.csv'

In [19]:
# Example usage
recommendation = recommend_species(
    lat=38.2274,
    lon=-85.8009,
    diameter_cm=1.2,
    native='naturally_occurring',
    city='Louisville',
    state='Kentucky',
    top_n=5
)

for species, count in recommendation:
    print(f"{species} (seen {count} times nearby)")

NameError: name 'recommend_species' is not defined

In [23]:
from tqdm import tqdm
from collections import defaultdict

def evaluate_recommender(X_scaled, df, model, top_k=5, sample_size=1000):
    correct = 0
    ranks = []

    for i in tqdm(range(sample_size)):
        x_query = X_scaled[i].reshape(1, -1)
        distances, indices = model.kneighbors(x_query)

        # exclude itself
        neighbor_indices = [idx for idx in indices[0] if idx != i][:top_k]
        true_species = df.iloc[i]['common_name']
        neighbor_species = df.iloc[neighbor_indices]['common_name'].tolist()

        if true_species in neighbor_species:
            correct += 1
            ranks.append(neighbor_species.index(true_species) + 1)
        else:
            ranks.append(0)

    hit_rate = correct / sample_size
    mean_rank = sum([1/r for r in ranks if r > 0]) / sample_size

    print(f"Top-{top_k} Hit Rate: {hit_rate:.4f}")
    print(f"Mean Reciprocal Rank: {mean_rank:.4f}")
    return hit_rate, mean_rank

# Run evaluation on a 1000-sample subset
evaluate_recommender(X_scaled, df, nn_model, top_k=5, sample_size=1000)

NameError: name 'X_scaled' is not defined

In [24]:
# Save scaler and model
import joblib
joblib.dump(scaler, 'scaler.joblib')
joblib.dump(nn_model, 'nn_model.joblib')

# Also save the dataframe with encoded columns (needed for categories and lookup)
df.to_pickle('tree_data.pkl')

print("Saved scaler, model and data!")

NameError: name 'scaler' is not defined

In [25]:
def get_common_locations_for_species(tree_name, top_n=10):
    """
    Given a tree common name, return the top N most frequent locations.
    """
    species_df = df[df['common_name'] == tree_name]

    if species_df.empty:
        return f"No records found for species: {tree_name}"

    # You can group by city/state or full address
    location_counts = species_df.groupby(['city', 'state']) \
                                .size().reset_index(name='count') \
                                .sort_values(by='count', ascending=False) \
                                .head(top_n)

    return location_counts

In [26]:
# Example tree name
tree_name = 'Bur oak'
top_locations = get_common_locations_for_species(tree_name, top_n=5)

print(f"Top locations where '{tree_name}' is commonly found:")
print(top_locations)

Top locations where 'Bur oak' is commonly found:
No records found for species: Bur oak
